<a href="https://colab.research.google.com/github/mmmovania/CUDA_Spring2023/blob/main/Week11/MergeSort_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-opm1rzyh
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-opm1rzyh
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=9a24aec5720a2ed83a40e38ac249b38431bf06e8872a052a9ed45f8a806775a3
  Stored in directory: /tmp/pip-ephem-wheel-cache-09evnpx4/wheels/db/c1/1f/a2bb07bbb4a1ce3c43921252aeafaa6205f08637e292496f04
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [2]:
%%cu
#include <stdio.h>
 
 //Ref: https://github.com/kevin-albert/cuda-mergesort/blob/master/mergesort.cu
 
inline cudaError_t checkCudaErr(cudaError_t err, const char* msg) {
	if (err != cudaSuccess) {
		fprintf(stderr, "CUDA Runtime error at %s: %s\n", msg, cudaGetErrorString(err));
	}
	return err;
}

__device__ void gpu_bottomUpMerge(int* source, int* dest, int start, int middle, int end) {
    int i = start;
    int j = middle;
    for (int k = start; k < end; k++) {
        if (i < middle && (j >= end || source[i] < source[j])) {
            dest[k] = source[i];
            i++;
        } else {
            dest[k] = source[j];
            j++;
        }
    }
}
 

__global__ void mergeSort( int *data, int *sorted_data, const int N, int width, int slices, int threads, int blocks) 
{	 
	int tid = threadIdx.x + blockIdx.x * blockDim.x;
  int start = width*tid*slices, middle, end;

	for (int slice = 0; slice < slices; slice++) {
  	if (start >= N)
    	break;

    middle = min(start + (width >> 1), N);
    end = min(start + width, N);
    gpu_bottomUpMerge(data, sorted_data, start, middle, end);
    start += width;
  }
}

int main() 
{ 
	const int N = 100;
	int  *data, *sorted_data; 

	// Allocate Unified Memory -- accessible from CPU or GPU
	checkCudaErr(cudaMallocManaged(&data, N*sizeof(int)), "cudaMallocManaged1");
	checkCudaErr(cudaMallocManaged(&sorted_data, N*sizeof(int)), "cudaMallocManaged2");
	 
	puts("Data before sorting: ");
	
	// fill in the memory with data
	for (int i=0; i<N; i++) {
		data[i] = rand() % 100 + 1;
		sorted_data[i] = 0;
	  printf("%d\n", data[i]);
	
	} 

  int blocksPerGrid = 32;
	int threadsPerBlock = 8;

	int* A = data;
  int* B = sorted_data;
	
	for (int width = 2; width < (N << 1); width <<= 1) 
  {
     long slices = N / ((threadsPerBlock) * width) + 1;
 
     // Actually call the kernel
     mergeSort<<<blocksPerGrid, threadsPerBlock>>>(A, B, N, width, slices, 
																								threadsPerBlock, blocksPerGrid);
 
     // Switch the input / output arrays instead of copying them around
     A = A == data ? sorted_data : data;
     B = B == data ? sorted_data : data;
  }
	cudaDeviceSynchronize();
  

	
	puts("Data after sorting: ");
	for(int i=0; i<N; ++i)
     printf("%d\n", sorted_data[i]);
	// free memory on the gpu side
	checkCudaErr( cudaFree( data ), "cudaFree1");
	checkCudaErr( cudaFree( sorted_data ), "cudaFree2"); 
	checkCudaErr( cudaDeviceReset(), "cudaDeviceReset");

	return 0;
}

Data before sorting: 
84
87
78
16
94
36
87
93
50
22
63
28
91
60
64
27
41
27
73
37
12
69
68
30
83
31
63
24
68
36
30
3
23
59
70
68
94
57
12
43
30
74
22
20
85
38
99
25
16
71
14
27
92
81
57
74
63
71
97
82
6
26
85
28
37
6
47
30
14
58
25
96
83
46
15
68
35
65
44
51
88
9
77
79
89
85
4
52
55
100
33
61
77
69
40
13
27
87
95
40
Data after sorting: 
3
4
6
6
9
12
12
13
14
14
15
16
16
20
22
22
23
24
25
25
26
27
27
27
27
28
28
30
30
30
30
31
33
35
36
36
37
37
38
40
40
41
43
44
46
47
50
51
52
55
57
57
58
59
60
61
63
63
63
64
65
68
68
68
68
69
69
70
71
71
73
74
74
77
77
78
79
81
82
83
83
84
85
85
85
87
87
87
88
89
91
92
93
94
94
95
96
97
99
100

